In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import sys
import csv
import math
from operator import itemgetter
import time

from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.externals import joblib
from sklearn.feature_selection import RFE, VarianceThreshold, SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression, mutual_info_classif, chi2
from sklearn import metrics
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import KBinsDiscretizer, scale

In [3]:
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier

In [4]:
import copy
import time

In [15]:
data = pd.read_csv('ivy_new.csv')
data.head()

,DepressionScore,AGE,US_Army)DMQMILIZ,BornInUS,Citizenship(DMDCITZN),EduLevelAdults(DMDEDUC2),MaritalStatus(DMDMARTL),(Eng/Span(SIALANG),#PPLinHousehold(DMDHHSIZ),#Kinds<=5yrs(DMDHHSZA),...,HowOftenDrankMilk,Race(RIDRETH3)_Asian,Race(RIDRETH3)_Black,Race(RIDRETH3)_MexicanAmerican,Race(RIDRETH3)_Other&MultiRacial,Race(RIDRETH3)_OtherHispanic,Race(RIDRETH3)_White,log_vitaminA,female,male
0,1,69,1,1,1,3,0,1,3,0,...,8,0,1,0,0,0,0,4.70,0,1
1,2,54,0,1,1,3,1,1,4,0,...,9,0,0,0,0,0,1,7.31,0,1
2,0,72,1,1,1,4,1,1,2,0,...,9,0,0,0,0,0,1,6.48,0,1
3,9,73,0,1,1,5,1,1,2,0,...,9,0,0,0,0,0,1,6.76,1,0
4,19,56,1,1,1,4,0,1,1,0,...,9,0,0,1,0,0,0,5.87,0,1


In [29]:
classy = pd.read_csv('classify.csv')
classy.head()

,DepressionScore,AGE,US_Army)DMQMILIZ,BornInUS,EduLevelAdults(DMDEDUC2),MaritalStatus(DMDMARTL),#PPLinHousehold(DMDHHSIZ),#Kinds<=5yrs(DMDHHSZA),#ofKinds6-17yrs(DMDHHSZB),#ofAdults>=60yrs(DMDHHSZE),...,HowOftenDrankMilk,Race(RIDRETH3)_Asian,Race(RIDRETH3)_Black,Race(RIDRETH3)_MexicanAmerican,Race(RIDRETH3)_Other&MultiRacial,Race(RIDRETH3)_OtherHispanic,Race(RIDRETH3)_White,log_vitaminA,female,male
0,0,72,1,1,4,1,2,0,0,2,...,9,0,0,0,0,0,1,6.48,0,1
1,0,26,0,1,5,0,3,0,0,0,...,9,0,0,0,0,0,1,7.26,1,0
2,0,18,0,1,3,0,7,0,2,2,...,8,0,0,1,0,0,0,4.53,0,1
3,0,50,0,0,5,1,3,0,1,0,...,7,1,0,0,0,0,0,6.31,0,1
4,0,28,1,0,4,1,2,0,0,0,...,6,1,0,0,0,0,0,8.75,0,1


In [30]:
class_header = list(classy.columns.values)
class_header

['DepressionScore',
 'AGE',
 'US_Army)DMQMILIZ',
 'BornInUS',
 'EduLevelAdults(DMDEDUC2)',
 'MaritalStatus(DMDMARTL)',
 '#PPLinHousehold(DMDHHSIZ)',
 '#Kinds<=5yrs(DMDHHSZA)',
 '#ofKinds6-17yrs(DMDHHSZB)',
 '#ofAdults>=60yrs(DMDHHSZE)',
 'HouseholdIncome(INDHHIN2)',
 'Energy(kcal)DR1TKCAL',
 'Protein(gm)DR1TPROT',
 'Carbs(gm)DR1TCARB',
 'Sugar(gm)DR1TSUGR',
 'fiber(gm)DR1TFIBE',
 ' TotalFat(gm)DR1TTFAT',
 'Cholesterol(mg)DR1TCHOL',
 'VitaminE(mg)DR1TATOC',
 'VitaminC(mg)DR1TVC',
 'VitaminD(D2 + D3)(mcg)DR1TVD',
 ' Calcium(mg)DR1TCALC',
 'Iron(mg)DR1TIRON',
 'Sodium(mg)DR1TSODI',
 'Caffeine(mg)DR1TCAFF',
 'FishEatenPast30Days1YesDRD360',
 'Avg #alcoholicDrinks/dayPast12mos(ALQ130)',
 '$spentAtSupermarket/grocery store(CBD070)',
 '$SpentOnNonfoodItems(CBD090)',
 '$SpentOnEatingOutPastMonth(CBD120)',
 '$SpentCarryout/deliveredFoodsPastMonth(CBD130)',
 'Past30dayMilkProductConsumption(DBQ197)',
 '#MealsNotHomePrepared(DBD895)',
 '#MealsFastFoodOrPizzaPlace(DBD900)',
 'MinutesOutdoors9-5Not

In [31]:
to_class = classy.iloc[:,1:]
to_class.head()

,AGE,US_Army)DMQMILIZ,BornInUS,EduLevelAdults(DMDEDUC2),MaritalStatus(DMDMARTL),#PPLinHousehold(DMDHHSIZ),#Kinds<=5yrs(DMDHHSZA),#ofKinds6-17yrs(DMDHHSZB),#ofAdults>=60yrs(DMDHHSZE),HouseholdIncome(INDHHIN2),...,HowOftenDrankMilk,Race(RIDRETH3)_Asian,Race(RIDRETH3)_Black,Race(RIDRETH3)_MexicanAmerican,Race(RIDRETH3)_Other&MultiRacial,Race(RIDRETH3)_OtherHispanic,Race(RIDRETH3)_White,log_vitaminA,female,male
0,72,1,1,4,1,2,0,0,2,10,...,9,0,0,0,0,0,1,6.48,0,1
1,26,0,1,5,0,3,0,0,0,15,...,9,0,0,0,0,0,1,7.26,1,0
2,18,0,1,3,0,7,0,2,2,5,...,8,0,0,1,0,0,0,4.53,0,1
3,50,0,0,5,1,3,0,1,0,15,...,7,1,0,0,0,0,0,6.31,0,1
4,28,1,0,4,1,2,0,0,0,7,...,6,1,0,0,0,0,0,8.75,0,1


In [32]:
target_class = classy['DepressionScore']
target_class.head()

0    0
1    0
2    0
3    0
4    0
Name: DepressionScore, dtype: int64

In [44]:
target_class.value_counts()

0    1713
1     205
Name: DepressionScore, dtype: int64

In [33]:
class_np = np.asarray(to_class)
target_np = np.asarray(target_class)

In [34]:
#random forest classifier feature selection

class_train, class_test, class_target_train, class_target_test = train_test_split(class_np, target_class, test_size=0.35)

rand_st = 1
clf= RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth= None, min_samples_split=3, random_state=rand_st)
clf.fit(class_np, target_class)
sel_idx = []
feature_importance = clf.feature_importances_
for x in feature_importance:
    if x >= np.mean(feature_importance):
        sel_idx.append(1)
    else:
        sel_idx.append(0)
temp=[]
temp_idx=[]
temp_del=[]
for i in range(len(class_np[0])):
    if sel_idx[i]==1:           #Selected Features get added to temp header
        temp.append(class_header[i+1])
        temp_idx.append(i)
    else:           #Indexes of non-selected features get added to delete array
        temp_del.append(i)
print('Selected:', temp)
print('Features (total/selected):', len(class_np[0]), len(temp))
print('\n')  

#Filter selected columns from original dataset 
header = class_header[0:1]
for field in temp:
    header.append(field)
new_np = np.delete(class_np, temp_del, axis=1) 
        

Selected: ['AGE', 'HouseholdIncome(INDHHIN2)', 'Energy(kcal)DR1TKCAL', 'Protein(gm)DR1TPROT', 'Carbs(gm)DR1TCARB', 'Sugar(gm)DR1TSUGR', 'fiber(gm)DR1TFIBE', ' TotalFat(gm)DR1TTFAT', 'Cholesterol(mg)DR1TCHOL', 'VitaminE(mg)DR1TATOC', 'VitaminC(mg)DR1TVC', 'VitaminD(D2 + D3)(mcg)DR1TVD', ' Calcium(mg)DR1TCALC', 'Iron(mg)DR1TIRON', 'Sodium(mg)DR1TSODI', 'Caffeine(mg)DR1TCAFF', '$spentAtSupermarket/grocery store(CBD070)', '$SpentOnEatingOutPastMonth(CBD120)', 'TypeOfWorkDoneLast weekOCD150', 'HoursWorkedLastWeekAtAllJobs(OCQ180)', 'NumberOfMonthsWorkingInMainJob(OCD270)', 'DurationOfLongestJob(months)OCD395', 'HoursWatchTVorVideosPast30Days(PAQ710)', 'HoursUseComputerPast30Days(PAQ715)', 'HowMuchSleepDoYouGet(hours)SLD010H', 'FirstSexAge(SXD031)', 'CurrentSelfReportedWeight(pounds)WHD020', 'AgeWhenHeaviestWeight(WHQ150)', 'log_vitaminA']
Features (total/selected): 56 29




In [26]:
feature_importance

array([0.02757838, 0.0018861 , 0.00352316, 0.01304053, 0.00732999,
       0.01145657, 0.00312774, 0.00557704, 0.00707417, 0.03316398,
       0.02133156, 0.02422528, 0.02337355, 0.02476703, 0.02272507,
       0.02460084, 0.02353273, 0.02678847, 0.02217774, 0.02259796,
       0.02200106, 0.02443517, 0.03123448, 0.02504761, 0.00339105,
       0.02814589, 0.01184065, 0.02372653, 0.01320572, 0.02255492,
       0.01122442, 0.00953336, 0.01488653, 0.01415875, 0.01033001,
       0.01261479, 0.02244596, 0.03118545, 0.01195309, 0.02721612,
       0.01824847, 0.02083584, 0.018483  , 0.04929541, 0.02337646,
       0.03845093, 0.02750469, 0.01301194, 0.00354808, 0.00373885,
       0.00247739, 0.00264143, 0.00519148, 0.00340689, 0.02466874,
       0.01268576, 0.01142518])

In [15]:
new_np.shape

(1918, 29)

In [27]:
sel=SelectKBest(chi2, k=4)
fit_mod=sel.fit(new_np, target_class)                                         #will throw error if any negative values in features, so turn off feature normalization, or switch to mutual_info_classif
print ('Univariate Feature Selection - Chi2: ')
sel_idx=fit_mod.get_support()

temp=[]
scores=fit_mod.scores_
for i in range(1, len(class_header)):            
        temp.append([class_header[i], float(scores[i-1])])

print('Ranked Features')
temp_sort=sorted(temp, key=itemgetter(1), reverse=True)
for i in range(len(temp_sort)):
    print(i, temp_sort[i][0], ':', temp_sort[i][1])
print('\n')

#2 Get lists of selected and non-selected features (names and indexes) #######
temp=[]
temp_idx=[]
temp_del=[]
for i in range(len(new_np[0])):
    if sel_idx[i]==1:                                                           #Selected Features get added to temp header
        temp.append(class_header[i+1])
        temp_idx.append(i)
    else:                                                                       #Indexes of non-selected features get added to delete array
        temp_del.append(i)
print('Selected', temp)
print('Features (total/selected):', len(new_np[0]), len(temp))
print('\n')

Univariate Feature Selection - Chi2: 


IndexError: index 29 is out of bounds for axis 0 with size 29

In [28]:
data_train, data_test, target_train, target_test = train_test_split(new_np, target_class, test_size=0.35)


In [ ]:
# SVC
scorers = {'Accuracy': 'accuracy', 'roc_auc': 'roc_auc'} 
clf=SVC(C = 1.0, kernel = 'linear', gamma = 'scale', probability = True, random_state = 1)
scores=cross_validate(clf, new_np, target_class, scoring=scorers, cv=5)

scores_Acc = scores['test_Accuracy']                                                                                                                                    
print("SVM Acc: %0.2f (+/- %0.2f)" % (scores_Acc.mean(), scores_Acc.std() * 2))                                                                                                    
scores_AUC= scores['test_roc_auc']                                                                     #Only works with binary classes, not multiclass                  
print("SVM AUC: %0.2f (+/- %0.2f)" % (scores_AUC.mean(), scores_AUC.std() * 2))                           
print("CV Runtime:", time.time()-start_ts)


In [35]:
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier

scorers = {'Accuracy': 'accuracy', 'roc_auc': 'roc_auc'}
# 
data_train, data_test, target_train, target_test = train_test_split(new_np, target_class, test_size=0.35)
start_ts=time.time()
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, loss='deviance', max_depth=3,  min_samples_split=3, random_state=1)
scores=cross_validate(clf, new_np, target_class, scoring=scorers, cv=5)

scores_Acc = scores['test_Accuracy']                                                                                                                                    
print("AdaBoost Acc: %0.2f (+/- %0.2f)" % (scores_Acc.mean(), scores_Acc.std() * 2))                                                                                                    
scores_AUC= scores['test_roc_auc']                                                                     #Only works with binary classes, not multiclass                  
print("AdaBoost AUC: %0.2f (+/- %0.2f)" % (scores_AUC.mean(), scores_AUC.std() * 2))                           
print("CV Runtime:", time.time()-start_ts)


AdaBoost Acc: 0.90 (+/- 0.01)
AdaBoost AUC: 0.78 (+/- 0.06)
CV Runtime: 1.347456932067871


In [41]:
new_np.shape

(1918, 29)

In [36]:
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier

scale_np = scale(new_np)

scorers = {'Accuracy': 'accuracy', 'roc_auc': 'roc_auc'}
# 
data_train, data_test, target_train, target_test = train_test_split(scale_np, target_class, test_size=0.35)
start_ts=time.time()
scale_clf=AdaBoostClassifier(n_estimators=100, learning_rate=0.1, base_estimator=None, random_state=rand_st)
scores=cross_validate(scale_clf, scale_np, target_class, scoring=scorers, cv=5)

scores_Acc = scores['test_Accuracy']                                                                                                                                    
print("AdaBoost Acc: %0.2f (+/- %0.2f)" % (scores_Acc.mean(), scores_Acc.std() * 2))                                                                                                    
scores_AUC= scores['test_roc_auc']                                                                     #Only works with binary classes, not multiclass                  
print("AdaBoost AUC: %0.2f (+/- %0.2f)" % (scores_AUC.mean(), scores_AUC.std() * 2))                           
print("CV Runtime:", time.time()-start_ts)

AdaBoost Acc: 0.89 (+/- 0.01)
AdaBoost AUC: 0.76 (+/- 0.07)
CV Runtime: 2.121515989303589


In [37]:
#gradient boosting classifier feature selection

class_train, class_test, class_target_train, class_target_test = train_test_split(class_np, target_class, test_size=0.35)

rand_st = 1
clf= RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth= None, min_samples_split=3, random_state=rand_st)
clf.fit(class_np, target_class)
sel_idx = []
feature_importance = clf.feature_importances_
for x in feature_importance:
    if x >= np.mean(feature_importance):
        sel_idx.append(1)
    else:
        sel_idx.append(0)
temp=[]
temp_idx=[]
temp_del=[]
for i in range(len(class_np[0])):
    if sel_idx[i]==1:                                   #Selected Features get added to temp header
        temp.append(class_header[i+1])
        temp_idx.append(i)
    else:                                                #Indexes of non-selected features get added to delete array
        temp_del.append(i)
print('Selected:', temp)
print('Features (total/selected):', len(class_np[0]), len(temp))
print('\n')  

#Filter selected columns from original dataset 
header = class_header[0:1]
for field in temp:
    header.append(field)
new_np = np.delete(class_np, temp_del, axis=1) 

Selected: ['AGE', 'HouseholdIncome(INDHHIN2)', 'Energy(kcal)DR1TKCAL', 'Protein(gm)DR1TPROT', 'Carbs(gm)DR1TCARB', 'Sugar(gm)DR1TSUGR', 'fiber(gm)DR1TFIBE', ' TotalFat(gm)DR1TTFAT', 'Cholesterol(mg)DR1TCHOL', 'VitaminE(mg)DR1TATOC', 'VitaminC(mg)DR1TVC', 'VitaminD(D2 + D3)(mcg)DR1TVD', ' Calcium(mg)DR1TCALC', 'Iron(mg)DR1TIRON', 'Sodium(mg)DR1TSODI', 'Caffeine(mg)DR1TCAFF', '$spentAtSupermarket/grocery store(CBD070)', '$SpentOnEatingOutPastMonth(CBD120)', 'TypeOfWorkDoneLast weekOCD150', 'HoursWorkedLastWeekAtAllJobs(OCQ180)', 'NumberOfMonthsWorkingInMainJob(OCD270)', 'DurationOfLongestJob(months)OCD395', 'HoursWatchTVorVideosPast30Days(PAQ710)', 'HoursUseComputerPast30Days(PAQ715)', 'HowMuchSleepDoYouGet(hours)SLD010H', 'FirstSexAge(SXD031)', 'CurrentSelfReportedWeight(pounds)WHD020', 'AgeWhenHeaviestWeight(WHQ150)', 'log_vitaminA']
Features (total/selected): 56 29




In [ ]:
##set kernal as linear so  it produces coefficients for FS

clf =SVC(C= 1.0, kernel='linear', gamma ='scale', probability = True, random_state = 1)#set kernal as linear so  it produces coefficients for FS 
sel = SelectFromModel(clf, prefit=False, threshold='-np.inf', max_features=None)                                                           #to select only based on max_features, set to integer value and set threshold=-np.inf
 #to select only based on max_features, set to integer value and set threshold=-np.inf
print ('Wrapper Select: ')
fit_mod=sel.fit(new_np, target_class)    
sel_idx=fit_mod.get_support()


temp=[]
scores=fit_mod.scores_
for i in range(1, len(class_header)): 
    temp.append([class_header[i], float(scores[i-feat_start])])

print('Ranked Features')
temp_sort=sorted(temp, key=itemgetter(1), reverse=True)
for i in range(len(temp_sort)):
    
    print(i, temp_sort[i][0], ':', temp_sort[i][1])
print('\n')

Wrapper Select: 


In [38]:
#random forest classifier feature selection with new set

class_train, class_test, class_target_train, class_target_test = train_test_split(new_np, target_class, test_size=0.35)

rand_st = 1
clf= RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth= None, min_samples_split=3, random_state=1)
clf.fit(new_np, target_class)
sel_idx = []
feature_importance = clf.feature_importances_
for x in feature_importance:
    if x >= np.mean(feature_importance):
        sel_idx.append(1)
    else:
        sel_idx.append(0)
temp=[]
temp_idx=[]
temp_del=[]
for i in range(len(new_np[0])):
    if sel_idx[i]==1:           #Selected Features get added to temp header
        temp.append(class_header[i+1])
        temp_idx.append(i)
    else:           #Indexes of non-selected features get added to delete array
        temp_del.append(i)
print('Selected:', temp)
print('Features (total/selected):', len(new_np[0]), len(temp))
print('\n')  

#Filter selected columns from original dataset 
header = class_header[0:1]
for field in temp:
    header.append(field)
small_np = np.delete(new_np, temp_del, axis=1) 

Selected: ['AGE', 'US_Army)DMQMILIZ', 'EduLevelAdults(DMDEDUC2)', 'fiber(gm)DR1TFIBE', 'VitaminC(mg)DR1TVC', 'VitaminD(D2 + D3)(mcg)DR1TVD', 'FishEatenPast30Days1YesDRD360', '$spentAtSupermarket/grocery store(CBD070)']
Features (total/selected): 29 8




In [39]:
# Filter selected columns from original dataset #########
header = header[0:1]
for field in temp:
    class_header.append(field)
smaller_np = np.delete(small_np, temp_del, axis=1) 

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  """


In [40]:
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier

#scale_np = scale(smaller_np)

scorers = {'Accuracy': 'accuracy', 'roc_auc': 'roc_auc'}
# 
data_train, data_test, target_train, target_test = train_test_split(smaller_np, target_class, test_size=0.35)
start_ts=time.time()
scale_clf=AdaBoostClassifier(n_estimators=100, learning_rate=0.1, base_estimator=None, random_state=rand_st)
scores=cross_validate(scale_clf, smaller_np, target_class, scoring=scorers, cv=5)

scores_Acc = scores['test_Accuracy']                                                                                                                                    
print("AdaBoost Acc: %0.2f (+/- %0.2f)" % (scores_Acc.mean(), scores_Acc.std() * 2))                                                                                                    
scores_AUC= scores['test_roc_auc']                                                                     #Only works with binary classes, not multiclass                  
print("AdaBoost AUC: %0.2f (+/- %0.2f)" % (scores_AUC.mean(), scores_AUC.std() * 2))                           
print("CV Runtime:", time.time()-start_ts)

AdaBoost Acc: 0.89 (+/- 0.00)
AdaBoost AUC: 0.67 (+/- 0.08)
CV Runtime: 0.9832019805908203
